In [4]:
import pandas as pd
from sklearn import preprocessing
from xgboost import XGBClassifier
import optuna
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


In [42]:
train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')

In [13]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96294 entries, 0 to 96293
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            96294 non-null  object 
 1   대출금액          96294 non-null  int64  
 2   대출기간          96294 non-null  object 
 3   근로기간          96294 non-null  object 
 4   주택소유상태        96294 non-null  object 
 5   연간소득          96294 non-null  int64  
 6   부채_대비_소득_비율   96294 non-null  float64
 7   총계좌수          96294 non-null  int64  
 8   대출목적          96294 non-null  object 
 9   최근_2년간_연체_횟수  96294 non-null  int64  
 10  총상환원금         96294 non-null  int64  
 11  총상환이자         96294 non-null  float64
 12  총연체금액         96294 non-null  float64
 13  연체계좌수         96294 non-null  float64
 14  대출등급          96294 non-null  object 
dtypes: float64(4), int64(5), object(6)
memory usage: 11.0+ MB


대출기간

In [43]:
train_data["대출기간"].value_counts()
# train["범죄발생지"].value_counts().values

data_frequency = {' 36 months':2, ' 60 months':1}
data_frequency.get("대출기간")

In [44]:
#train
data_frequency.get(train_data.loc[1, "대출기간"])  # '차도':13

for loc in range(0, train_data.shape[0]):
    encd = data_frequency.get(train_data.loc[loc, "대출기간"])
    train_data.loc[loc, "대출기간(encode)"] = encd
train_data = train_data.drop(columns='대출기간')
train_data.head(2)     # 84406, 27

#test
data_frequency.get(test_data.loc[1, "대출기간"])  # '차도':13

for loc in range(0, test_data.shape[0]):
    encd = data_frequency.get(test_data.loc[loc, "대출기간"])
    test_data.loc[loc, "대출기간(encode)"] = encd
test_data = test_data.drop(columns='대출기간')
test_data.head(2)     # 84406, 27

,ID,대출금액,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출기간(encode)
0,TEST_00000,16800000,8 years,MORTGAGE,132000000,19.64,12,주택 개선,0,394692,146604.0,0.0,0.0,2.0
1,TEST_00001,8400000,5 years,RENT,89971200,15.84,25,부채 통합,0,0,0.0,0.0,0.0,2.0


근로기간

In [45]:
train_data['근로기간'] = train_data['근로기간'].str.extract(r'(\d+)')
train_data['근로기간']  =train_data['근로기간'].fillna(0)
train_data['근로기간']  =train_data['근로기간'].astype(int)

test_data['근로기간'] = test_data['근로기간'].str.extract(r'(\d+)')
test_data['근로기간']  =test_data['근로기간'].fillna(0)
test_data['근로기간']  =test_data['근로기간'].astype(int)

원핫 인코딩

In [46]:
train_data = pd.get_dummies(train_data, columns = ['주택소유상태', '대출목적'])
test_data = pd.get_dummies(test_data, columns = ['주택소유상태', '대출목적'])

train_data.head(2)

,ID,대출금액,근로기간,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,...,대출목적_소규모 사업,대출목적_신용 카드,대출목적_의료,대출목적_이사,대출목적_자동차,대출목적_재생 에너지,대출목적_주요 구매,대출목적_주택,대출목적_주택 개선,대출목적_휴가
0,TRAIN_00000,12480000,6,72000000,18.90,15,0,0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1,TRAIN_00001,14400000,10,130800000,22.33,21,0,373572,234060.0,0.0,...,False,False,False,False,False,False,False,False,True,False


대출등급

In [47]:
label_encoder_target = preprocessing.LabelEncoder() 
train_data['대출등급'] = label_encoder_target.fit_transform(train_data['대출등급'] )

oversampling


In [21]:
train_data_feature = train_data.drop(columns='ID')
train_data_target = train_data['대출등급']
train_data_feature = train_data_feature.drop(columns='대출등급')

In [22]:
train_data_x, valid_data_x, train_data_y, valid_data_y= train_test_split(train_data_feature,train_data_target,test_size=0.2, shuffle=True,random_state=34)

In [23]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=19)
train_data_x_smote, train_data_y_smote = smote.fit_resample(train_data_x, train_data_y)
print(train_data_x_smote.shape, train_data_y_smote.shape)

(161399, 27) (161399,)


train

In [25]:
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 1000, 10000),
        'max_depth': trial.suggest_int('max_depth', 8, 20),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.1, 5),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1),
        'subsample': trial.suggest_float('subsample', 0.5, 1)
    }

    model = XGBClassifier(**param, random_state=1004)
    model.fit(train_data_x_smote, train_data_y_smote)
    preds = model.predict(valid_data_x)
    accuracy = accuracy_score(valid_data_y, preds)
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)  # 예를 들어 100번의 시도

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2024-01-30 01:49:53,296] A new study created in memory with name: no-name-670bbbac-b69a-4941-8bbf-1db2a3ab8454
[I 2024-01-30 01:54:32,155] Trial 0 finished with value: 0.8573134638350901 and parameters: {'n_estimators': 2743, 'max_depth': 18, 'min_child_weight': 6, 'gamma': 1.6213030634026273, 'learning_rate': 0.056569138269197995, 'colsample_bytree': 0.6630264041721299, 'reg_lambda': 0.8516099073009213, 'reg_alpha': 0.6523383867674757, 'subsample': 0.8441511392310261}. Best is trial 0 with value: 0.8573134638350901.
[I 2024-01-30 02:01:34,275] Trial 1 finished with value: 0.848278726829015 and parameters: {'n_estimators': 4189, 'max_depth': 11, 'min_child_weight': 2, 'gamma': 3.161225542258085, 'learning_rate': 0.08384532167357274, 'colsample_bytree': 0.8832467049856755, 'reg_lambda': 0.6052400662018332, 'reg_alpha': 0.8573867099156004, 'subsample': 0.7511307257567541}. Best is trial 0 with value: 0.8573134638350901.
[I 2024-01-30 02:05:19,117] Trial 2 finished with value: 0.855755

Number of finished trials: 30
Best trial: {'n_estimators': 3060, 'max_depth': 18, 'min_child_weight': 8, 'gamma': 1.3711914364420739, 'learning_rate': 0.07179118539189841, 'colsample_bytree': 0.596380817284563, 'reg_lambda': 0.21357461370886702, 'reg_alpha': 0.3547883252823349, 'subsample': 0.9046998743176695}


In [26]:
model = XGBClassifier(**study.best_trial.params)
model.fit(train_data_feature, train_data_target)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.596380817284563, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=1.3711914364420739,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.07179118539189841,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=18, max_leaves=None,
              min_child_weight=8, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=3060, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [39]:
train_data_feature.columns

Index(['대출금액', '근로기간', '연간소득', '부채_대비_소득_비율', '총계좌수', '최근_2년간_연체_횟수', '총상환원금',
       '총상환이자', '총연체금액', '연체계좌수', '대출기간(encode)', '주택소유상태_ANY',
       '주택소유상태_MORTGAGE', '주택소유상태_OWN', '주택소유상태_RENT', '대출목적_기타', '대출목적_부채 통합',
       '대출목적_소규모 사업', '대출목적_신용 카드', '대출목적_의료', '대출목적_이사', '대출목적_자동차',
       '대출목적_재생 에너지', '대출목적_주요 구매', '대출목적_주택', '대출목적_주택 개선', '대출목적_휴가'],
      dtype='object')

In [55]:
test_data['주택소유상태_ANY'] = False
test_data = test_data.drop(columns='ID')


In [67]:
test_data.columns

Index(['대출금액', '근로기간', '연간소득', '부채_대비_소득_비율', '총계좌수', '최근_2년간_연체_횟수', '총상환원금',
       '총상환이자', '총연체금액', '연체계좌수', '대출기간(encode)', '주택소유상태_ANY',
       '주택소유상태_MORTGAGE', '주택소유상태_OWN', '주택소유상태_RENT', '대출목적_기타', '대출목적_부채 통합',
       '대출목적_소규모 사업', '대출목적_신용 카드', '대출목적_의료', '대출목적_이사', '대출목적_자동차',
       '대출목적_재생 에너지', '대출목적_주요 구매', '대출목적_주택', '대출목적_주택 개선', '대출목적_휴가'],
      dtype='object')

In [66]:
desired_position = 11 # 이동하길 원하는 위치(0부터 시작)
test_data.insert(desired_position, '주택소유상태_ANY', test_data.pop('주택소유상태_ANY'))


In [68]:
# test_data = test_data.drop(columns='주택소유상태_ANY')
test_pred = model.predict(test_data)

In [69]:
test_pred_decoded = label_encoder_target.inverse_transform(test_pred)
submission_data = pd.read_csv('./sample_submission.csv')
submission_data['대출등급'] = test_pred_decoded
submission_data.to_csv('./xgboost.csv', index=False)

In [70]:
test_pred_decoded

array(['B', 'B', 'A', ..., 'D', 'C', 'A'], dtype=object)